In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
batting_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/Batting.csv'
salary_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/Salaries.csv'
people_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/People.csv'
fielding_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/Fielding.csv'
pitching_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/Pitching.csv'

batting_df = pd.read_csv(batting_url)
salary_df = pd.read_csv(salary_url)
people_df = pd.read_csv(people_url)
fielding_df = pd.read_csv(fielding_url)
pitching_df = pd.read_csv(pitching_url)

## Introduction

- This project aims to train a model on MLB salary data, in order to make predictions. This model will take as input a number of offensive, defensive, and biographical features (such as position, age, season number), and output an estimate for the player's salary for that year.
- This is the first version of the model, just to 'get it off the ground' so to speak. I am excited to see how it fares, what features work and don't work for prediction, and especially excited for future iterations. The largest issue I see this model facing is that the MLB has an 'odd' salary structure as is, primarily in terms of how arbitration works. Due to this, I believe that a "season_num" feature will be extremely important for the model. I believe in training the model the way I intend, the predictions will be coming primarily from the season_num feature (which I will have to create), position feature, and some combination of offensive features. If a player is a short stop in their 7th year and hits over 30 home runs, the model should estimate a very high salary for that player.
- Another challenge is in the data wrangling journey. The source of the data (the Society for American Baseball Research's Lahman Baseball Database) is quite well kept, thus the cleaning will not be too difficult. However, the data is stored in a number of different tables, and hence, there will be a quite involved process in getting the attributes I need from the different tables (i.e., hits, at-bats, home runs are all located in the "Batting.csv" file, and I will of course need the salary feature from "Salaries.csv", and defensive features will have to be retrieved from the "Fielding.csv" file). This will be relatively straightforward however, just a number of joins.
- Finally, the features that I want to train the model on are not always "ready out of the box". This isn't my first analysis/modelling project working with the dataset, and I know that the Batting.csv file does not have any rate statistics (batting average, OBP, etc.) so we will have to create them ourselves, furthermore, as salaries have changed over the years, I believe it will be very important to normalize the salaries relative to the mean and standard deviation of that seasons salaries. The dataset also does not contain a "season number" feature, so that will have to be created manually.
- With the project goal, files, and challenges ready, we are ready to begin.

## Analyzing Features

- This section will analyze what features are available in the datasets so we can make decisions as to what features we wish to include.

In [3]:
# fielding_df will let us see what defensive metrics we can include.
fielding_df

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
0,aardsda01,2004,1,SFN,NL,P,11,0.0,32.0,0,0,0.0,0,NaN,NaN,NaN,NaN,NaN
1,aardsda01,2006,1,CHN,NL,P,45,0.0,159.0,1,5,0.0,1,NaN,NaN,NaN,NaN,NaN
2,aardsda01,2007,1,CHA,AL,P,25,0.0,97.0,2,4,1.0,0,NaN,NaN,NaN,NaN,NaN
3,aardsda01,2008,1,BOS,AL,P,47,0.0,146.0,3,6,0.0,0,NaN,NaN,NaN,NaN,NaN
4,aardsda01,2009,1,SEA,AL,P,73,0.0,214.0,2,5,0.0,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153651,zwilldu01,1915,1,CHF,FL,OF,148,NaN,NaN,356,20,8.0,6,NaN,NaN,NaN,NaN,NaN
153652,zwilldu01,1916,1,CHN,NL,OF,10,NaN,NaN,11,0,0.0,0,NaN,NaN,NaN,NaN,NaN
153653,zychto01,2015,1,SEA,AL,P,13,1.0,55.0,0,3,0.0,0,NaN,NaN,NaN,NaN,NaN
153654,zychto01,2016,1,SEA,AL,P,12,0.0,41.0,0,0,1.0,0,NaN,NaN,NaN,NaN,NaN


- If possible, ZR is a more robust defensive metric than, for example, errors, but there appears to be a large number of incomplete logs. It seems unlikely we will be able to use this metric.

In [4]:
# Seeing if we can utilize ZR as a metric for quantifying player defensive ability.

fielding_df[fielding_df['ZR']>-100]

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
621,adamsdo01,1969,1,CHA,AL,C,4,3.0,78.0,9,2,0.0,0,1.0,0.0,0.0,0.0,0.0
854,adlesda01,1963,1,HOU,NL,C,6,0.0,46.0,8,0,1.0,0,1.0,1.0,1.0,0.0,0.0
855,adlesda01,1964,1,HOU,NL,C,3,2.0,63.0,11,2,0.0,0,0.0,2.0,2.0,2.0,0.0
856,adlesda01,1965,1,HOU,NL,C,13,10.0,246.0,51,5,0.0,1,1.0,3.0,4.0,3.0,0.0
857,adlesda01,1966,1,HOU,NL,C,1,0.0,21.0,11,0,0.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153405,zimmeje01,1967,1,MIN,AL,C,104,81.0,2077.0,572,44,5.0,7,1.0,20.0,29.0,26.0,3.0
153406,zimmeje01,1968,1,MIN,AL,C,24,18.0,435.0,109,7,1.0,0,0.0,1.0,6.0,5.0,0.0
153620,zupofr01,1957,1,BAL,AL,C,8,1.0,78.0,20,1,2.0,0,1.0,0.0,2.0,1.0,0.0
153621,zupofr01,1958,1,BAL,AL,C,1,0.0,12.0,4,0,0.0,0,0.0,1.0,0.0,0.0,0.0


- Using a filter (any record with a ZR above -100), we see that there are only 1169 records, compared to the 153656 that is initially stored, we will not be using this metric.
- We will (relatively unfortunately) use a combination of fielding percentage (which we will have to create), errors, and position played for our defensive metrics.

In [5]:
batting_df.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'G_batting', 'AB',
       'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP',
       'SH', 'SF', 'GIDP', 'G_old'],
      dtype='object')

- For the offensive side of things, we will use a combination of 'H', 'AB', '2B', '3B', 'HR', 'RBI', 'SB', 'BB', 'IBB', 'HBP' features to create our own bevy of features: Batting average (AVG), on-base percentage (OBP), on-base-plus-slugging (OPS), and slugging (SLG). These are the traditional rate statistics, and we may as well create all of them as it will save time later to experiment with different combinations for model training.

## What features to use:

### Offensive Categories

* Batting Average: Traditional statistic that only recently was "moved away from", a solid indicator of a player's overall hitting ability
* Home Runs: "Eye-popping" statistic that historically has been linked with high-paying contracts and good offensive production
* OPS: A very good encapsulative stat for offensive production, but since this is a relatively new stat, it may not necessarily be a great training attribute.
* OBP: Similar to OPS, except it does not account for SLG. It may be wise to use batting average, OBP, and Home Runs as our training features, as they are encapsulative of almost everything that has been historically valued when it comes to offensive production.
* SB: A historically valued stat to evaluate a player's speed and athleticism on the base paths.

### Defensive Categories

* Position: Extremely important, premium defensive positions get paid more on average. It is important, however, to find a means to distinguish a good and bad defender.
* Errors: Since the dataset does not have a lot of other defensive metrics, errors will have to suffice. This is also not horrible, as in the past this has been used by front offices to determine a player's defensive capabilities.
* Fielding percentage: See "Errors"

### Miscellaneous

* Batting Handedness: This one is more speculative, I plan on training a model with and without including batting handedness and comparing performance.
* Age: This is more important if we were to try and predict the full contract at which a player will sign, but since we are focusing primarily on a season-by-season salary prediction, age will likely not be as important an attribute.
* Season: This is an extremely important attribute for baseball especially (the attribute is the number of seasons the player has played, inclusive). This is due to the arbitration system. The model should be able to pick up on the trends with salaries and season numbers.

## Feature creation

In [6]:
# Adding a "season" feature, which will denote what year the player is in during each record. Very important for distinguishing whether a player is in arbitration or free agency.

salary_df['season'] = salary_df.groupby('playerID').cumcount() + 1

In [7]:
# Finding when salaries began being tracked in the dataset.

salary_df['yearID'].min()

1985

Salaries are only tracked from 1985 onward in the dataset, thus we will have to pare down the other datasets to only include data from 1985 onwards, as salary is the target variable here.

Game-plan here is to determine which features I want to include, and then joining the appropriate dataframes on playerID and yearID, resulting in an aggregated dataset with the important features and the target variable.

### Creating batting features

- Traditional triple slash statistics
    - AVG: Batting average, hits per at-bats
    - OBP: On-base percentage, percentage of times a player safely reaches base per plate appearance
    - PA: Plate appearances, a sum of a players at-bats, walks, hit-by-pitches, and sacrifice flies.
    - SLG: Slugging, a weighted batting average, i.e., a home run is counted four times compared to a single.
    - OPS: On-base plus slugging, a sum of SLG and OBP.

In [8]:
# Since team should have little correlation with a player's salary, we will summarize their
# overall contributions for both teams by sorting the values by games played, then by choosing
# the first teamID as seen below. This is because we need there to be one record per player 
# per season. A similar process will need to be done for fielding position, as there are 
# multiple records per season if a player played multiple seasons.

batting_df = batting_df[batting_df['AB']>100]
batting_df_sorted = batting_df.sort_values(by=['playerID','yearID','G'], ascending = [True, True, False])

batting_df = batting_df_sorted.groupby(['playerID','yearID'],as_index=False).agg({
    'teamID':'first',
    'G':'sum',
    'AB':'sum',
    'H':'sum',
    'HR':'sum',
    '2B':'sum',
    '3B':'sum',
    'HBP':'sum',
    'BB':'sum',
    'SB':'sum',
    'SF':'sum',
    'IBB':'sum'
})

# Creating slashline statistics.

batting_df['AVG'] = round(batting_df['H']/batting_df['AB'],3)
batting_df['SLG'] = round(((batting_df['H']-batting_df['2B']-batting_df['3B']-batting_df['HR'])+2*batting_df['2B']+3*batting_df['3B']+4*batting_df['HR'])/batting_df['AB'],3)
batting_df['PA'] = round(batting_df['AB']+batting_df['BB']+batting_df['HBP']+batting_df['SF'],3)
batting_df['OBP'] = round((batting_df['H']+batting_df['BB']+batting_df['HBP'])/batting_df['PA'],3)
batting_df['OPS'] = round(batting_df['OBP'] + batting_df['SLG'],3)


batting_df.head()

,playerID,yearID,teamID,G,AB,H,HR,2B,3B,HBP,BB,SB,SF,IBB,AVG,SLG,PA,OBP,OPS
0,aaronha01,1954,ML1,122,468,131,13,27,6,3.0,28,2.0,4.0,0.0,0.280,0.447,503.0,0.322,0.769
1,aaronha01,1955,ML1,153,602,189,27,37,9,3.0,49,3.0,4.0,5.0,0.314,0.540,658.0,0.366,0.906
2,aaronha01,1956,ML1,153,609,200,26,34,14,2.0,37,2.0,7.0,6.0,0.328,0.558,655.0,0.365,0.923
3,aaronha01,1957,ML1,151,615,198,44,27,6,0.0,57,1.0,3.0,15.0,0.322,0.600,675.0,0.378,0.978
4,aaronha01,1958,ML1,153,601,196,30,34,4,1.0,59,4.0,3.0,16.0,0.326,0.546,664.0,0.386,0.932


In [9]:
#salary_df_sorted = salary_df.sort_values(by=['playerID','yearID','G'], ascending = [True, True, False])

#batting_df = batting_df_sorted.groupby(['playerID','yearID'],as_index=False).agg({
#    'teamID':'first',
#    'G':'sum',
#    'AB':'sum',
#    'H':'sum',
#    'HR':'sum',
#    '2B':'sum',
#    '3B':'sum',
#    'HBP':'sum',
#    'BB':'sum',
#    'SB':'sum',
#    'SF':'sum',
#    'IBB':'sum'
#})

### Creating fielding features
- FP: Fielding percentage, number of putouts + number of assists / total number of opportunities
- Opps: Opportunities, sum of putouts, assists, and errors.
- Note: We will not include pitchers in the model training as their salaries are understandably not much related to their offensive or fielding prowess.

In [10]:
fielding_df = fielding_df[fielding_df['POS']!='P']

fielding_df['Opps'] = fielding_df['A'] + fielding_df['E'] + fielding_df['PO']

fielding_df = fielding_df[fielding_df['Opps']>0]

fielding_df['FP'] = (fielding_df['A'] + fielding_df['PO'])/fielding_df['Opps']
fielding_df.head()


fielding_sorted = fielding_df.sort_values(['playerID','yearID','G'])
fielding_primary_pos = fielding_sorted.drop_duplicates(subset=['playerID','yearID'], keep = 'first')

/tmp/ipykernel_17/414867414.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fielding_df['Opps'] = fielding_df['A'] + fielding_df['E'] + fielding_df['PO']


## Normalizing Salary

In [11]:
yearly_salary = pd.DataFrame()
yearly_salary['mean'] = salary_df.groupby(['yearID'])['salary'].mean()
yearly_salary['sd'] = salary_df.groupby(['yearID'])['salary'].std()
yearly_salary

,mean,sd
yearID,,
1985,4.762994e+05,3.433209e+05
1986,4.171470e+05,3.940176e+05
1987,4.347295e+05,4.311822e+05
1988,4.531711e+05,4.531581e+05
1989,5.063231e+05,5.380454e+05
1990,5.119737e+05,5.524405e+05
1991,8.949612e+05,9.398494e+05
1992,1.047521e+06,1.180646e+06
1993,9.769666e+05,1.284734e+06


In [12]:
salary_df = pd.merge(salary_df,yearly_salary, on=['yearID'])

In [13]:
salary_df['salary_norm'] = (salary_df['salary']-salary_df['mean'])/salary_df['sd']

## Joining dataframes

In [14]:
cols = ['playerID','yearID','POS','FP','E']
fielding = fielding_primary_pos[cols]
cols = ['playerID','yearID','AVG','OBP','HR','SLG','AB']
batting = batting_df[cols].set_index('playerID')

comb = pd.merge(fielding, batting, on=['playerID','yearID'])
comb.head()

salary = salary_df[['playerID','yearID','salary_norm','season']]

comb2 = pd.merge(comb, salary, on=['playerID','yearID'])
comb2 = comb2[comb2['AB']>100]

## Creating Model

In [15]:
comb2

,playerID,yearID,POS,FP,E,AVG,OBP,HR,SLG,AB,salary_norm,season
0,abbotje01,1998,OF,0.970588,4.0,0.279,0.298,12,0.492,244,-0.606519,1
1,abbotje01,2000,OF,0.980952,2.0,0.274,0.343,3,0.395,215,-0.690417,3
2,abbotku01,1994,SS,0.965675,15.0,0.249,0.291,9,0.394,345,-0.695350,2
3,abbotku01,1995,SS,0.958515,19.0,0.255,0.318,17,0.452,420,-0.552789,3
4,abbotku01,1996,2B,1.000000,0.0,0.253,0.307,8,0.428,320,-0.502965,4
...,...,...,...,...,...,...,...,...,...,...,...,...
11489,zuletju01,2001,1B,0.990991,2.0,0.217,0.288,6,0.415,106,-0.715285,1
11490,zuninmi01,2014,C,0.995450,5.0,0.199,0.254,22,0.404,438,-0.674319,1
11491,zuninmi01,2015,C,0.994172,5.0,0.174,0.230,11,0.300,350,-0.686098,2
11492,zupcibo01,1992,OF,0.976744,6.0,0.276,0.322,3,0.352,392,-0.794921,2
